In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
import datetime
import import_ipynb
from tensorboard.plugins.hparams import api as hp
from dataloader import get_LDW_X_df, get_y_df, data_prep

2023-12-17 20:41:25.229234: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 20:41:25.260596: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 20:41:25.261488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-17 20:41:25.892981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


importing Jupyter notebook from dataloader.ipynb
Tensorflow version: 2.12.0


In [4]:
rm -rf ./logs/

In [5]:
csv_filepath = "../../data/csv/plant_data.csv"

In [6]:
X = get_LDW_X_df(csv_filepath)
y = get_y_df(csv_filepath, 'LDW_g')

In [7]:
HP_NUM_UNITS = hp.HParam("num_units", hp.Discrete([16, 32, 64, 128]))
HP_DROPOUT = hp.HParam("dropout", hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam("optimizer", hp.Discrete(['adam', 'sgd']))
HP_ACTIVATION = hp.HParam("activation", hp.Discrete(['linear', 'relu', 'sigmoid']))

METRIC_ACCURACY = 'loss'

In [8]:
with tf.summary.create_file_writer('../../logs/hyperparam_tuning/repression/LDW').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_ACTIVATION],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name="Test Accuracy")]
    )

2023-12-17 20:41:27.133575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-17 20:41:27.156022: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
X_train, X_test, y_train, y_test = data_prep(X, y)

In [10]:
def LDW_model(hparams):
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_input', input_shape = (11,),),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(hparams[HP_NUM_UNITS], hparams[HP_ACTIVATION], name='layers_dense2'),
            tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(1, name='layers_dense3')
        ])

        model.compile (
            optimizer=hparams[HP_OPTIMIZER],
            loss='mae',
        )

        model.fit (
            X_train,
            y_train,
            epochs=100,
            validation_split=0.2,
            callbacks=[
                tf.keras.callbacks.TensorBoard(log_dir=log_dir),
                hp.KerasCallback(log_dir, hparams)
            ]
        )

        accuracy = model.evaluate(X_test, y_test)
        return accuracy

        

In [11]:
log_dir = "../../logs/hyperparam_tuning/regression/LDW/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [12]:
def run_model(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = LDW_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [13]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for activation in HP_ACTIVATION.domain.values:
            for optimizer in HP_OPTIMIZER.domain.values:
                hparams = {
                    HP_ACTIVATION: activation,
                    HP_DROPOUT: dropout,
                    HP_NUM_UNITS: num_units,
                    HP_OPTIMIZER: optimizer
                }

                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                run_model(log_dir + run_name, hparams)
                session_num += 1

--- Starting trial: run-0
{'activation': 'linear', 'dropout': 0.1, 'num_units': 16, 'optimizer': 'adam'}
Epoch 1/100
11/11 [==============================] - 1s 12ms/step - loss: 3.3583 - val_loss: 3.1751
Epoch 2/100
11/11 [==============================] - 0s 3ms/step - loss: 3.2518 - val_loss: 3.1495
Epoch 3/100
11/11 [==============================] - 0s 3ms/step - loss: 3.1928 - val_loss: 3.1077
Epoch 4/100
11/11 [==============================] - 0s 3ms/step - loss: 3.1298 - val_loss: 3.0519
Epoch 5/100
11/11 [==============================] - 0s 3ms/step - loss: 3.0660 - val_loss: 2.9901
Epoch 6/100
11/11 [==============================] - 0s 3ms/step - loss: 2.9934 - val_loss: 2.9100
Epoch 7/100
11/11 [==============================] - 0s 3ms/step - loss: 2.9196 - val_loss: 2.8228
Epoch 8/100
11/11 [==============================] - 0s 3ms/step - loss: 2.8396 - val_loss: 2.7405
Epoch 9/100
11/11 [==============================] - 0s 3ms/step - loss: 2.7439 - val_loss: 2.6481
Epo